In [1]:
import os

## TASK 1 - FASTQ

In [9]:
def find_fastq_files(root="."): 
    for dirpath, dirnames, filenames in os.walk(root):
        for filename in filenames:
            if filename.lower().endswith(('.fastq','.fq')):
                yield os.path.join(dirpath, filename)

In [12]:
## my logic for this sub task is a function that recieves the file path as input, we traverse through the file
## line by line until we reach end of file(the header returns a empty string). We also keep track of the sequence 
## which we then compare to the minimum length, if it's higher then we add one to our counter and in either case
## we add 1 to our total counter which keeps track of total "reads" or "DNA fragments" which we then use to calculate 
## the percentage. This approach is recursive and also doesn't require large- memory overhead. 

def calc_pct_long_reads(fastq_path, min_len=30):
    total = 0
    over = 0
    with open(fastq_path, 'r') as f:
        while True:
            header = f.readline()
            if not header:
                break
            seq = f.readline().strip()
            f.readline()
            f.readline()
            total += 1
            if len(seq) > min_len:
                over += 1
    if total > 0:
        pct = (over/total) * 100
    else:
        pct = 0
    return total, over, pct
    

In [11]:
## runnning the function in current directory
for fq in find_fastq_files('.'):
    total, over, pct = calc_pct_long_reads(fq, 30)
    print(f"{fq}: {pct:.2f}% ({over}/{total}) reads > 30 nt")

./sample_files/fastq/read2/Sample_R2.fastq: 83.60% (989/1183) reads > 30 nt
./sample_files/fastq/read1/Sample_R1.fastq: 80.64% (954/1183) reads > 30 nt
./sample_files/fastq/read1/.ipynb_checkpoints/Sample_R1-checkpoint.fastq: 80.64% (954/1183) reads > 30 nt


## TASK 1 - FASTA

In [18]:
## I use a dictionary to keep count of the sequences, the logic is pretty simple we read each line 
## and identify if it's the start of a sequence or not if it is we look for it in the dictionary and add one to it
## counter, returning the final dictionary. We then go through the items in dict and sort the top 10 most frequent ones
def parse_fasta(path):
    counts = {}
    seq = ''

    with open(path,'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if seq:
                    counts[seq] = counts.get(seq,0) + 1
                seq = ''
            else:
                seq += line
        if seq:
            counts[seq] = counts.get(seq,0) + 1
    return counts

In [17]:
fasta_path = "sample_files/fasta/sample.fasta"
counts_dict = parse_fasta(fasta_path)

top10 = sorted(
    counts_dict.items(), 
    key=lambda pair: pair[1], 
    reverse=True
)[:10]
## results
print("Count\tSequence")
for seq, cnt in top10:
    print(f"{cnt}\t{seq}")

Count	Sequence
28	CGCGCAGGCTGAAGTAGTTACGCCCCTGTAAAGGAATCTATGGACAATGGAACGAACA
27	TGTTCTGAGTCAAATGATATTAACTATGCTTATCACATATTATAAAAGACCGTGGACATTCATCTTTAGTGTGTCTCCCTCTTCCTACT
22	CTCAATCTGCCAAGACCATAGATCCTCTCTTACTGTCAGCTCATCCGGTGAGGCC
21	CCTGTTGCTGACTCAAGACATTAGTGAGAAATAAGACTTCTGCGATGCTCACCACTGCAATTGCTCATGCAAAATTGCGTTTAACAGG
17	ATTGCGAATTCCGCCTGTGTCCCCCACACGAGCGTGAATCGTGGCTAGAAGTTCAGCCCCTCTTAGCACAGAGTGAG
17	TTTCAGCTGTCTTTTAAGCAGAAGCGATTTGTCCAACAAAAACAACGCTGTTTACGAA
8	GACACAAACACCGTGGCTCAACCTAATCCTATTAGAGCCGAAAAGGCGAGGATGCTGATTGAGTAGGTATCTGGA
8	TCACGCAGACAACGAACTGTGTCTGGATCAAAGACATCCGATAAGGCGATTCGTCTAGAAGGGTTACACAGTTGGGACCGGTAG
5	TGCTTAAACTCATGATAGTCCCTGAGTAAACTGGTTGCGACACGGCTCCCG
5	TGTGCAGAATATAATGTAAAAAAAACAGGACCCGGCTCTGTGCCGTTGGCCTGCGCGGTACTCATGTTAGTTTTCCGACTCCGACTTAT


## Task 1 - Annotation

In [19]:
## my logic for this sub task is to first parse the GTF annotation file once,
## extracting only the “gene” features and pulling out chromosome, start, end, and gene_name.
## I store these as tuples in a dict then sort each chromosome’s list 
## by start coordinate and also keep a parallel list of just the start positions.
## This gives me a O(logn) lookup structure
## Next, I read the coordinates file line by line For each entry,
## I check if the chromosome exists in my index, if so, I use bisect_right on the starts list
## to get an insertion point i. 
## Finally, I write out a new TSV where each input line (chrom, pos) is appended with the
## gene_name. This approach loads the GTF into memory only once,
## uses pure built‑ins, and handles millions of lookups efficiently with minimal overhead.
import re
import bisect

def build_gene_index(gtf_path):
    """
    Parse only the “gene” entries from the GTF and build:
      • index: { chrom: [(start,end,gene_name), …] }
      • starts: { chrom: [start1, start2, …] }  # for bisect
    """
    index = {}
    attr_pattern = re.compile(r'(\S+)\s+"([^"]+)"')
    with open(gtf_path) as gtf:
        for line in gtf:
            if line.startswith('#'):
                continue
            cols = line.rstrip('\n').split('\t')
            chrom, feature, start, end, attrs = cols[0], cols[2], int(cols[3]), int(cols[4]), cols[8]
            if feature != 'gene':
                continue
            # extract gene_name (fallback to gene_id if missing)
            attr_dict = {k:v for k,v in attr_pattern.findall(attrs)}
            gene = attr_dict.get('gene_name', attr_dict.get('gene_id', 'NA'))
            index.setdefault(chrom, []).append((start, end, gene))

    ## sort each chrom’s list and build a parallel list of starts
    starts = {}
    for chrom, ivals in index.items():
        ivals.sort(key=lambda x: x[0])
        starts[chrom] = [iv[0] for iv in ivals]
        index[chrom] = ivals

    return index, starts

# build it once
gtf_path = 'sample_files/gtf/hg19_annotations.gtf'
gene_index, gene_starts = build_gene_index(gtf_path)


In [22]:
def annotate_positions(coord_path, index, starts, out_path):
    """
    For each line “chrom<TAB>pos” in coord_path, 
    find which gene (if any) contains pos, and write:
      chrom<TAB>pos<TAB>gene_name/intergenic
    """
    with open(coord_path) as fin, open(out_path, 'w') as fout:
        for line in fin:
            chrom, pos = line.split('\t')
            pos = int(pos)
            gene_name = 'intergenic'
            if chrom in index:
                st_list = starts[chrom]
                ivals   = index[chrom]
                i = bisect.bisect_right(st_list, pos)
                if i:
                    s,e,name = ivals[i-1]
                    if e >= pos:
                        gene_name = name
            fout.write(f"{chrom}\t{pos}\t{gene_name}\n")

## running it
coords = 'sample_files/annotate/coordinates_to_annotate.txt'
out_file = 'coordinates_annotated.tsv'
annotate_positions(coords, gene_index, gene_starts, out_file)
print(f"Written annotated results to {out_file}")


Written annotated results to coordinates_annotated.tsv


## Task 2

In [25]:
## my logic for this sub task is to define 10 equal GC‐content bins based on the %gc value
## I initialize a dictionary to accumulate total coverage and count per bin.
## Then, I open the file and read it line by line with a CSV DictReader, stripping out any intervals with missing coverage.
## For each valid row, I convert the GC fraction and mean coverage to floats, compute which bin index it falls into
## (multiplying GC by 10 and capping at 9 for gc==1.0), and then add the coverage to that bin’s running sum and increment its counter.
## After processing every interval, I loop over the bins in order, compute the average coverage (sum/count) for each one,
## and print out the bin label alongside its mean target coverage. This approach uses only built‐ins, runs in one pass,
## and keeps memory usage small since I never load the entire file at once.```

import csv
bin_edges = [i/10 for i in range(11)]
labels = [f"{int(bin_edges[i]*100)}-{int(bin_edges[i+1]*100)}%" for i in range(len(bin_edges)-1)]

stats = {label: [0.0, 0] for label in labels}

with open('Example.hs_intervals.txt') as f:
    reader = csv.DictReader(f, delimiter='\t')
    for row in reader:
        # pull and strip both fields
        gc_str  = row['%gc'].strip()
        cov_str = row['mean_coverage'].strip()

        # if either is missing, skip this interval
        if not gc_str or not cov_str:
            continue

        gc  = float(gc_str)
        cov = float(cov_str)

        # assign to bin as before...
        idx = min(int(gc*10), 9)
        label = labels[idx]
        stats[label][0] += cov
        stats[label][1] += 1


## computing and print means
print("GC_bin\tmean_target_coverage")
for label in labels:
    total_cov, count = stats[label]
    if count > 0:
        mean_cov = total_cov / count
        print(f"{label}\t{mean_cov:.2f}")


GC_bin	mean_target_coverage
0-10%	0.00
10-20%	69.29
20-30%	77.93
30-40%	99.02
40-50%	101.28
50-60%	92.12
60-70%	78.93
70-80%	37.83
80-90%	10.28
